In [ ]:
import requests
from bs4 import BeautifulSoup,  SoupStrainer
import numpy as np
import sys
import datetime
import sqlite3

In [ ]:
def download(method, url, headers = {"user-agent":None}, param=None, verify=False, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url, params = param, data = data, headers=headers, verify=False)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            resp = download(method, url, param, data, timeout, maxretries-1)
            
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [ ]:
url = "http://www.1372.go.kr/contents/include.ccn?gSiteCode=2&gMenuCode=4&nMenuCode=13"

In [ ]:
len(url)

In [ ]:
db = list()
def scrape(url, page, db):
    data = {"startCount":page}
    html = download('post', url, data = data)
    dom = BeautifulSoup(html.text, 'lxml')
    tbody = dom.findAll('tbody')[1]
    foo = [_.text.split('\n') for _ in tbody.findAll('tr')]
    
    base = 'http://www.ccn.go.kr/contents'
    subject = [base + _.a['href'] for _ in tbody.findAll('tr')]
    
    for i, j in zip(foo, subject):
        temp = dict()
        temp['num'] = i[1]
        temp['gooboon'] = i[2]
        temp['itemCode'] = i[3]
        temp['title'] = i[4]
        temp['receiptDate'] = i[6]
        
        inside = download('get', j)
        inside_dom = BeautifulSoup(inside.text, 'lxml')
        tr = inside_dom.select("table[class^=boardView] > tbody > tr")

        temp['receiptNum'] = tr[0].select("td")[0].text
        temp['date'] = tr[0].select("td")[1].text
        temp['item'] = tr[1].select("td")[0].text
        temp['answerDate'] =  tr[3].select("td")[0].text
        question = inside_dom.findAll('div', {'class':'autocounsel_last_box_mid'})
        temp['question'] = question[0].text
        temp['answer'] = question[1].text

        db.append(temp)
    return db

In [ ]:
i = 0
db = list()
startTime = datetime.datetime.now()
while i < 145850 :
    db = scrape(url, i, db)
    i += 10
    if i % 1000 == 0:
        cur = datetime.datetime.now()
        c = startTime - cur
        print(i, 'th present counsel QA:', db[i-1]['title'], 'Time until now:', divmod(c.days * 86400 + c.seconds, 60))

In [ ]:
import json

In [ ]:
with open('temp_Counseling.json', 'w') as json_file:
    json.dump(db, json_file)

In [ ]:
len(db)

In [ ]:
with open('temp_Counseling.json') as json_file:
    data = json.load(json_file)

In [ ]:
del db